# Measurement of pixel properties of 3D images

#### Code created by Deniz Bekat (*deniz.bekat@crick.ac.uk*)

This python Notebook is a beginner-friendly, accessible way to measure clearing efficacy in 3d images; it is designed to:
- import an image and visualise with napari
- do simple pre-processing to remove noise
- perform simple segmentation to detect the ROI of the image (in this case, your 3D biological sample)
- measure the mean pixel intensity and standard deviation of the pixel intensities to deduce clearing efficacy




### **Importing libraries**

Python contains several **modules and packages** that provide the tools to analyse our images; we can use the code ``` import ``` to use them within this code:

In [ ]:
#importing required modules

import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from skimage import io, filters
import seaborn as sns

import napari


### **Importing your image with ```skimage.io```**

Your image can be imported with one of the imported modules, skimage (short for scikit image); this is a very common way to import images into your code!

**NOTE**: this might take a while if you have a very large image file, so don't be alarmed :)

In [ ]:
image_path = ""
im_read = io.imread(image_path) #uses skimage.io to import the image as a numpy array


im_read.shape #check the shape of your image; using this code requires CZXY formatting (you can do this with np.tranpose)

### **OPTIONAL: Choosing your nuclear channel**
If you have more than one channel, then you should reduce it down to a 1-channel image using ```im_read[n]```, where n is the channel  (preferably nuclear/autofluorescence)

**NOTE**: Python counts from 0, so if you have 3 channels, then it is numbered 0-2, etc.

In [ ]:
im_read = im_read[0]

### **Visualisation with napari**
We use `napari` as an open-source, interactive viewer of 3D images. The code below opens a window and presents the raw data (if you have it downloaded - [link attached](https://napari.org/stable/#))

In [ ]:
#visualisation of image in napari

viewer = napari.Viewer() #opens a napari window
raw_img_layer = viewer.add_image(im_read) #puts the raw data onto the viewer

### **Pre-processing**

To remove any noise (from image artefacts, etc.), we apply **Gaussian blur** using `skimage.filters.gaussian`:

In [ ]:
# Gaussian blur to remove noise 
im_gauss = filters.gaussian(im_read, sigma=(0.5,2,2))


### **Image Thresholding**

The area of interest (in this case, your sample) can be separated from the background using one of **several pre-prepared algorithms**

However, not each algorithm will work for every image; so we can use `filters.try_all_threshold` to see what works best on a small subset of your data

After that, you can choose the proper threshold with `filters.threshold_[algorithm name]`

In [ ]:
# display various options for gray level thresholding; choose what is most appropriate
fig, ax = filters.try_all_threshold(im_gauss[int(im_gauss.shape[0] / 3)], figsize=(8, 10), verbose=False)
plt.tight_layout()

In [ ]:
# thresholding image in 3D before labelling; make sure to edit the threshold algorithm by what suits your data best!
thresh = filters.threshold_mean(im_gauss)
thresh_img = im_gauss >= thresh

# adding the thresholding to napari to view thresholding throughout the 3D image (check if there are any large errors)
thresh_img_view = viewer.add_image(thresh_img)

### Measuring properties per stack

The key part of measuring clearing efficacy **is to measure key parameters and see how they change as you go deeper into the sample**.

There are certain pre-built algorithms for this (such as ```skimage.measure.label```), which uses **instance segmentation** however we want to use **semantic segmentation**; so we measure things manually per slice instead:

In [ ]:
# measuring properties per image slice; plot across z to see how properties change with image thickness
mean_data = []
std_data = []
for z in range(0, len(im_read)): #checks through every slice to 
    
    slice = im_read[z]
    binary_mask = thresh_img[z]

    if np.any(thresh_img):  # avoid empty mask
        mean = np.mean(slice[binary_mask])
        std = np.std(slice[binary_mask])
    else:
        mean, std = np.nan, np.nan  # handle empty slices

    mean_data.append(mean)
    std_data.append(std)


step_size_Z = 3 #size of the z-dimension of your voxels

df = pd.DataFrame({
    "Slice": np.arange(len(mean)) + 1, # +1 accounts for Python counting from 0, while slices start from 1
    "Mean Intensity": mean_data,
    "Std Dev": std_data
})

df['Imaging depth'] = df['Slice'] * step_size_Z 
df['Type'] = 'LAV' #change variable to the type of microscope used
df['file_name'] = image_path

df.tail() #quick overview of the pandas DataFrame

### Visualise and save data

We can use `seaborn` to plot lineplots of the mean pixel intensity and the standard deviation:

- if your image is of an uncleared sample, we expect these should degrade over time as light scattering loses biological information
- if your image is a cleared sample, then we expect these measurements to stay relatively constant throughout


We can then save that data to a .csv file, and do any type of analysis on the data!

In [ ]:
sns.scatterplot(data=df, x='Imaging depth', y='Mean Intensity') #quick plot of mean int and std dev in order to assess algorithm efficacy
plt.show()
sns.scatterplot(data=df, x='Imaging depth', y='Std Dev')

In [ ]:
#saving pandas dataframe as .csv file

df.to_csv('filename.csv') #saves your data as a .csv file, so that you can either continue to analyse in Python, or in R, or some softwares!